<a href="https://colab.research.google.com/github/PinskayaNika/MMO_Pinskaya/blob/main/%D0%A0%D0%9A2/RK2_Pinskaya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Рубежный контроль №2**
**Студент:** Пинская Ника Марковна  
**Группа:** ИУ5-21М   
**Вариант (номер по списку группы):** 9  
**Тема:** Методы обработки текстов 

**Решение задачи классификации текстов**  

Необходимо решить задачу классификации текстов на основе любого выбранного Вами датасета (кроме примера, который рассматривался в лекции). Классификация может быть бинарной или многоклассовой. Целевой признак из выбранного Вами датасета может иметь любой физический смысл, примером является задача анализа тональности текста.

Необходимо сформировать два варианта векторизации признаков - на основе CountVectorizer и на основе TfidfVectorizer.

В качестве классификаторов необходимо использовать два классификатора по варианту для группы ИУ5-21М:

**Классификатор №1:** LogisticRegression  
**Классификатор №2:** Multinomial Naive Bayes - MNB

In [1]:
#Импорт библиотек

import numpy as np
import pandas as pd
from typing import Dict, Tuple
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, median_absolute_error, r2_score 
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
import seaborn as sns
from collections import Counter
from sklearn.model_selection import train_test_split

#from sklearn.datasets import fetch_20newsgroups
import matplotlib.pyplot as plt

%matplotlib inline 
sns.set(style="ticks")

In [2]:
# Подключение к gogle диску
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Вывод содержимого папки на диске
import os
data_root = '/content/drive/MyDrive/MMO'
print(os.listdir(data_root))

['pulitzer-circulation-data.csv', 'mmsa-icu-beds2.csv', 'movies.csv.zip', 'movies2.csv.zip', 'movies1.csv.zip', 'avengers.csv.zip', 'comma-survey.csv.zip', 'Tweets.csv.zip']


In [4]:
#Распаковка архива с датасетом
!unzip /content/drive/MyDrive/MMO/Tweets.csv.zip

Archive:  /content/drive/MyDrive/MMO/Tweets.csv.zip
  inflating: Tweets.csv              


In [5]:
# Загрузка данных
data = pd.read_csv("Tweets.csv")
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [6]:
data.shape

(14640, 15)

In [7]:
# data = data.overview.fillna(' ')

In [8]:
# Сформируем общий словарь для обучения моделей из обучающей и тестовой выборки
vocab_list = data['text'].tolist()
vocab_list[1:10]

["@VirginAmerica plus you've added commercials to the experience... tacky.",
 "@VirginAmerica I didn't today... Must mean I need to take another trip!",
 '@VirginAmerica it\'s really aggressive to blast obnoxious "entertainment" in your guests\' faces &amp; they have little recourse',
 "@VirginAmerica and it's a really big bad thing about it",
 "@VirginAmerica seriously would pay $30 a flight for seats that didn't have this playing.\nit's really the only bad thing about flying VA",
 '@VirginAmerica yes, nearly every time I fly VX this “ear worm” won’t go away :)',
 '@VirginAmerica Really missed a prime opportunity for Men Without Hats parody, there. https://t.co/mWpG7grEZP',
 "@virginamerica Well, I didn't…but NOW I DO! :-D",
 "@VirginAmerica it was amazing, and arrived an hour early. You're too good to me."]

In [9]:
vocabVect = CountVectorizer()
vocabVect.fit(vocab_list)
corpusVocab = vocabVect.vocabulary_
print('Количество сформированных признаков - {}'.format(len(corpusVocab)))

Количество сформированных признаков - 15051


In [10]:
for i in list(corpusVocab)[1:10]:
    print('{}={}'.format(i, corpusVocab[i]))

what=14551
dhepburn=4804
said=11646
plus=10438
you=14944
ve=14193
added=1965
commercials=4030
to=13326


In [11]:
# categories = ["rec.motorcycles", "rec.sport.baseball", "sci.electronics","sci.med"]
# newsgroups = fetch_20newsgroups(subset='train', categories=categories)
# data = newsgroups['data']

In [12]:
def accuracy_score_for_classes(
    y_true: np.ndarray, 
    y_pred: np.ndarray) -> Dict[int, float]:
    """
    Вычисление метрики accuracy для каждого класса
    y_true - истинные значения классов
    y_pred - предсказанные значения классов
    Возвращает словарь: ключ - метка класса, 
    значение - Accuracy для данного класса
    """
    # Для удобства фильтрации сформируем Pandas DataFrame 
    d = {'t': y_true, 'p': y_pred}
    df = pd.DataFrame(data=d)
    # Метки классов
    classes = np.unique(y_true)
    # Результирующий словарь
    res = dict()
    # Перебор меток классов
    for c in classes:
        # отфильтруем данные, которые соответствуют 
        # текущей метке класса в истинных значениях
        temp_data_flt = df[df['t']==c]
        # расчет accuracy для заданной метки класса
        temp_acc = accuracy_score(
            temp_data_flt['t'].values, 
            temp_data_flt['p'].values)
        # сохранение результата в словарь
        res[c] = temp_acc
    return res

def print_accuracy_score_for_classes(
    y_true: np.ndarray, 
    y_pred: np.ndarray):
    """
    Вывод метрики accuracy для каждого класса
    """
    accs = accuracy_score_for_classes(y_true, y_pred)
    if len(accs)>0:
        print('Метка \t Accuracy')
    for i in accs:
        print('{} \t {}'.format(i, accs[i]))

In [13]:
# x = data.overview.fillna(' ')
# vocabVect = CountVectorizer()
# vocabVect.fit(data[x])
# corpusVocab = vocabVect.vocabulary_
# print('Количество сформированных признаков - {}'.format(len(corpusVocab)))

In [14]:
# for i in list(corpusVocab)[1:10]:
#     print('{}={}'.format(i, corpusVocab[i]))

**Использование класса *CountVectorizer***  
Подсчитывает количество слов словаря, входящих в данный текст.

In [15]:
test_features = vocabVect.transform(data)
test_features

<15x15051 sparse matrix of type '<class 'numpy.int64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [16]:
test_features.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [17]:
# Размер нулевой строки
len(test_features.todense()[0].getA1())

15051

In [18]:
# Непустые значения нулевой строки
[i for i in test_features.todense()[0].getA1() if i>0]

[]

In [19]:
vocabVect.get_feature_names()[100:120]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['10d',
 '10f',
 '10hrs',
 '10m',
 '10min',
 '10mins',
 '10p',
 '10pm',
 '10th',
 '10tmthvfdc',
 '10voucherwhatajoke',
 '10x',
 '10x9x17',
 '10yr',
 '11',
 '110',
 '1101',
 '1102',
 '1106',
 '1108']

**Использование класса *TfidfVectorizer***  
Вычисляет специфичность текста в корпусе текстов на основе метрики *TF-IDF*.

In [20]:
tfidfv = TfidfVectorizer(ngram_range=(1,3))
tfidf_ngram_features = tfidfv.fit_transform(vocab_list)
# tfidf_ngram_features = tfidfv.fit_transform(data['overview'])
tfidf_ngram_features

<14640x293223 sparse matrix of type '<class 'numpy.float64'>'
	with 684985 stored elements in Compressed Sparse Row format>

In [ ]:
tfidf_ngram_features.todense()

In [ ]:
# Размер нулевой строки
len(tfidf_ngram_features.todense()[0].getA1())

In [ ]:
# Непустые значения нулевой строки
[i for i in tfidf_ngram_features.todense()[0].getA1() if i>0]

**Решение задачи анализа тональности текста**  
С использованием кросс-валидации попробуем применить к корпусу текстов различные варианты векторизации и классификации.

In [ ]:
def VectorizeAndClassify(vectorizers_list, classifiers_list):
    for v in vectorizers_list:
        for c in classifiers_list:
            pipeline1 = Pipeline([("vectorizer", v), ("classifier", c)])
            score = cross_val_score(pipeline1, data['text'], data['airline'], scoring='accuracy', cv=3).mean()
            print('Векторизация - {}'.format(v))
            print('Модель для классификации - {}'.format(c))
            print('Accuracy = {}'.format(score))
            print('===========================')

In [ ]:
vectorizers_list = [CountVectorizer(vocabulary = corpusVocab), TfidfVectorizer(vocabulary = corpusVocab)]
classifiers_list = [LogisticRegression(), MultinomialNB()]
VectorizeAndClassify(vectorizers_list, classifiers_list)

**Разделим выборку на обучающую и тестовую**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['airline'], test_size=0.5, random_state=1)


In [ ]:
def sentiment(v, c):
    model = Pipeline(
        [("vectorizer", v), 
         ("classifier", c)])
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print_accuracy_score_for_classes(y_test, y_pred)

In [ ]:
sentiment(TfidfVectorizer(), LogisticRegression(C=5.0))


In [ ]:
sentiment(TfidfVectorizer(ngram_range=(1,3)), LogisticRegression(C=5.0))


In [ ]:
sentiment(TfidfVectorizer(ngram_range=(2,3)), LogisticRegression(C=5.0))


In [ ]:
sentiment(TfidfVectorizer(ngram_range=(1,4)), LogisticRegression(C=5.0))


In [ ]:
sentiment(TfidfVectorizer(ngram_range=(2,4)), LogisticRegression(C=5.0))


**Вывод:**  
Как видно из результатов, лучшую точность показал *CountVectorizer* и *MultinomialNB*.  
Точность составила 97,4%.